In [ ]:
import pandas as pd 

In [ ]:
df = pd.read_excel("/content/New Microsoft Excel Worksheet.xlsx", sheet_name = 'Sheet2')

In [ ]:
df.head(3)

,Home_Logout_t,User,x,y,User.1,Room
0,2022-07-01 00:00:00,000: SYSTEM,Alive Check,000: -,0,SYSTEM
1,2022-07-01 00:06:22,083: ROOM 330,Call,000: -,83,ROOM 330
2,2022-07-01 00:06:54,083: ROOM 330,Accept,000: -,83,ROOM 330


In [ ]:
pv = df.pivot_table(index='Room', columns='Home_Logout_t', values='x', aggfunc = len )
pv.head(3)

Home_Logout_t,2022-07-01 00:00:00,2022-07-01 00:06:22,2022-07-01 00:06:54,2022-07-01 00:07:26,2022-07-01 00:12:39,2022-07-01 00:12:47,2022-07-01 00:14:19,2022-07-01 00:14:24,2022-07-01 00:15:15,2022-07-01 00:42:02,...,2022-07-31 22:27:57,2022-07-31 22:28:14,2022-07-31 22:31:41,2022-07-31 22:32:01,2022-07-31 22:57:46,2022-07-31 22:57:51,2022-07-31 22:58:23,2022-07-31 23:00:00,2022-07-31 23:28:45,2022-07-31 23:29:34
Room,,,,,,,,,,,,,,,,,,,,,
ADDRESS 109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADDRESS 110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADDRESS 111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pv = df.set_index('Room')
pv

,Home_Logout_t,User,x,y,User.1
Room,,,,,
SYSTEM,2022-07-01 00:00:00,000: SYSTEM,Alive Check,000: -,0
ROOM 330,2022-07-01 00:06:22,083: ROOM 330,Call,000: -,83
ROOM 330,2022-07-01 00:06:54,083: ROOM 330,Accept,000: -,83
ROOM 330,2022-07-01 00:07:26,083: ROOM 330,Reset,000: -,83
ROOM 312,2022-07-01 00:12:39,045: ROOM 312,Call,000: -,45
...,...,...,...,...,...
ROOM 204,2022-07-31 22:57:51,015: ROOM 204,Accept,000: -,15
ROOM 204,2022-07-31 22:58:23,015: ROOM 204,Reset,000: -,15
SYSTEM,2022-07-31 23:00:00,000: SYSTEM,Alive Check,000: -,0


In [ ]:
file_name = 'excel.xlsx'
pv.to_excel(file_name)

In [ ]:
!pip install pandasql
from pandasql import sqldf

In [ ]:
pip install fugue[sql]

In [ ]:
from fugue_notebook import setup
setup ()

<IPython.core.display.Javascript object>

In [ ]:
from fugue import NativeExecutionEngine, DataFrames, PandasDataFrame, Schema, Transformer, IterableDataFrame, ArrayDataFrame
from fugue_sql import FugueSQLWorkflow
from IPython.display import display, HTML
from typing import Iterable, Dict, Any
from datetime import datetime
import pandas as pd

In [ ]:
class Pivot(Transformer):
    def get_output_schema(self, df):
        fields = [f for f in df.schema.fields if not f.name.startswith("_")]
        col = self.params.get_or_throw("col",str)
        return Schema(fields)+f"date:datetime,{col}:int"
    
    def on_init(self, df):
        self.value_col = self.params.get_or_throw("col",str)
        self.cols = [f for f in df.schema.names if not f.startswith("_")]
        self.date_cols = {f:datetime.strptime(f[1:], '%m_%d_%y') 
                          for f in df.schema.names if f.startswith("_")}
        
    def transform(self, df):
        def get_rows():
            for row in df.as_dict_iterable():
                res = [row[k] for k in self.cols]
                for k,v in self.date_cols.items():
                    yield res + [v,int(row[k])]
        return IterableDataFrame(get_rows(), self.output_schema)

In [ ]:
%%fsql
SELECT * WHERE Room = 'ROOM 330' LIMIT 100
death = TRANSFORM USING Pivot(col="x")
OUTPUT USING pprint

FugueSQLError: ignored

In [ ]:
import pandas as pd 

In [ ]:
df = pd.read_excel("/content/Nursing_Calls_Mr_Ahmed.xlsx", sheet_name = 'calls')

In [ ]:
df2 = pd.read_excel("/content/Nursing_Calls_Mr_Ahmed.xlsx", sheet_name = 'reset')

In [ ]:
dummy_frame = pd.read_excel("/content/Nursing_Calls_Mr_Ahmed - reset.xlsx", sheet_name = 'Main')

In [ ]:
dummy_frame.head(5)

,dummy


In [ ]:
dummy_frame = dummy_frame['x']

In [ ]:
#Pre-defining order to check in the dataframe
correct_order = ["Call","Accept","Reset"]

In [ ]:
dummy_frame = pd.DataFrame(columns=["dummy"])

In [ ]:
#For Loop Based on length of the order (Triplets in this case)
for i in range(0,len(dummy_frame),len(correct_order)):

    #Check if the order is matched
    if correct_order != dummy_frame["x"][i:i+3].tolist():
        for j in range(len(correct_order)):

            #Check for the incorrect value in the triplet
            if correct_order[j] != dummy_frame["x"][i:i+3].tolist()[j]:
                print ("Value at index:",i+j,"is incorrect.")
                print ("Current Value:",dummy_frame["x"][i:i+3].tolist()[j],"Correct Value is:",correct_order[j])

KeyError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Nasaaem Team Training.zip

unzip:  cannot find or open /content/drive/MyDrive/Nasaaem, /content/drive/MyDrive/Nasaaem.zip or /content/drive/MyDrive/Nasaaem.ZIP.
